# 07 — Tight End Market Inefficiency

**Research Question:** Why do late-round TEs provide exceptional value? When do tight ends peak? How do red zone efficiency and role (blocking vs. receiving) affect value?

**Analysis:**
1. Career arc analysis (peak age, decline rate)
2. Draft ROI by round
3. Top 10 bargains (late-round steals like George Kittle, Dalton Schultz)
4. Top 10 busts (first-round disappointments)
5. Red zone efficiency vs. salary scatter plot
6. Blocking vs. receiving TE value comparison

**Outputs:**
- Career arc: Value score by years of experience
- Draft ROI: Average value by draft round (TE-specific)
- Top bargains and busts visualizations
- Red zone efficiency analysis
- Blocking vs. receiving TE comparison

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
from pathlib import Path

# Set data directory relative to notebook location
DATA_DIR = Path('../data')

import plotly.io as pio
pio.renderers.default = 'notebook'

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

---
## 1. Load Data

In [ ]:
# Load scored player-seasons
scored = pd.read_parquet(DATA_DIR / 'scored.parquet', engine='fastparquet')
print(f"Scored data: {scored.shape[0]:,} player-seasons")

# Load rosters for draft and age information
rosters = pd.read_parquet(DATA_DIR / 'rosters.parquet', engine='fastparquet')
print(f"Rosters data: {rosters.shape[0]:,} player-week records")

# Get unique player draft info
draft_info = rosters[rosters['draft_number'].notna()][['player_id', 'player_name', 'draft_number', 'draft_club', 'rookie_year']].drop_duplicates('player_id').copy()
draft_info['draft_number'] = pd.to_numeric(draft_info['draft_number'], errors='coerce')
draft_info['draft_round'] = ((draft_info['draft_number'] - 1) // 32 + 1).astype('Int64')
draft_info['draft_round'] = draft_info['draft_round'].clip(upper=7)

print(f"\nDrafted players with info: {len(draft_info):,}")

---
## 2. Filter to Tight Ends

In [ ]:
# Filter scored data to TEs only
te_scored = scored[scored['pos_group'] == 'TE'].copy()
print(f"TE player-seasons: {te_scored.shape[0]:,}")
print(f"Unique TE players: {te_scored['player_id'].nunique():,}")
print(f"Seasons covered: {sorted(te_scored['season'].unique())}")

# Merge with draft info
te_df = te_scored.merge(draft_info, on=['player_id', 'player_name'], how='left')
print(f"\nTEs with draft info: {te_df['draft_number'].notna().sum():,} player-seasons")

# Calculate years since draft
te_df['years_since_draft'] = te_df['season'] - te_df['rookie_year']

# Show sample
te_df[['player_name', 'season', 'draft_round', 'draft_number', 'years_since_draft', 'value_score', 'apy_cap_pct']].head(10)

---
## 3. Career Arc Analysis (Performance by Experience)

In [ ]:
# Filter to TEs with draft info and valid years_since_draft
te_career = te_df[te_df['years_since_draft'].notna() & (te_df['years_since_draft'] >= 0)].copy()
te_career['years_since_draft'] = te_career['years_since_draft'].astype(int)

# Cap at 10 years (small sample after that)
te_career = te_career[te_career['years_since_draft'] <= 10].copy()

# Aggregate by years since draft
career_arc = te_career.groupby('years_since_draft').agg(
    avg_value=('value_score', 'mean'),
    median_value=('value_score', 'median'),
    avg_performance=('performance_zscore', 'mean'),
    avg_salary_pct=('apy_cap_pct', 'mean'),
    count=('value_score', 'size')
).reset_index()

print("Career Arc Summary:")
print(career_arc)

# Line chart: Value score over career
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=career_arc['years_since_draft'],
    y=career_arc['avg_value'],
    mode='lines+markers',
    name='Avg Value Score',
    marker=dict(size=10),
    line=dict(width=3),
    hovertemplate='<b>Year %{x}</b><br>Avg Value: %{y:.2f}<br>N=%{customdata:,}<extra></extra>',
    customdata=career_arc['count']
))

fig.update_layout(
    title='<b>TE Career Arc: Value Score by Years of Experience</b><br><sub>When Do Tight Ends Peak?</sub>',
    xaxis_title='Years Since Draft',
    yaxis_title='Average Value Score',
    width=900,
    height=500,
    hovermode='x'
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
fig.show()

# Save visualization
output_dir = Path('../article/images/te_market_inefficiency')
output_dir.mkdir(parents=True, exist_ok=True)
fig.write_image(output_dir / 'career_arc.png', width=900, height=500)

---
## 4. Draft ROI by Round (TE-Specific)

In [ ]:
# Filter to rookie contract years (first 4 seasons)
te_rookie = te_career[(te_career['years_since_draft'] >= 0) & (te_career['years_since_draft'] <= 3)].copy()
print(f"TE rookie contract seasons: {te_rookie.shape[0]:,}")

# Aggregate by draft round
te_draft_roi = te_rookie.groupby('draft_round').agg(
    avg_value=('value_score', 'mean'),
    median_value=('value_score', 'median'),
    avg_performance=('performance_zscore', 'mean'),
    avg_salary_pct=('apy_cap_pct', 'mean'),
    count=('value_score', 'size'),
    pct_bargains=('is_bargain', lambda x: x.sum() / len(x) * 100 if len(x) > 0 else 0)
).reset_index()

print("\nTE Draft ROI by Round:")
print(te_draft_roi)

# Bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=te_draft_roi['draft_round'],
    y=te_draft_roi['avg_value'],
    text=te_draft_roi['avg_value'].round(2),
    textposition='outside',
    marker_color=['green' if v > 0.3 else 'orange' if v > 0 else 'red' for v in te_draft_roi['avg_value']],
    hovertemplate='<b>Round %{x}</b><br>Avg Value: %{y:.2f}<br>N=%{customdata[0]:,}<br>% Bargains: %{customdata[1]:.1f}%<extra></extra>',
    customdata=te_draft_roi[['count', 'pct_bargains']]
))

fig.update_layout(
    title='<b>TE Draft ROI: Average Value by Round</b><br><sub>Rookie Contract Years Only | Why Round 5 Is the Best-Kept Secret</sub>',
    xaxis_title='Draft Round',
    yaxis_title='Average Value Score',
    width=800,
    height=500,
    showlegend=False
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
fig.show()

# Save visualization
fig.write_image(output_dir / 'draft_roi_by_round.png', width=800, height=500)

---
## 5. Top 10 TE Bargains (Late-Round Steals)

In [ ]:
# Aggregate by player over their career
te_player_agg = te_df[te_df['draft_number'].notna()].groupby(['player_name', 'draft_round', 'draft_number']).agg(
    avg_value=('value_score', 'mean'),
    total_seasons=('season', 'size'),
    avg_performance=('performance_zscore', 'mean'),
    avg_salary_pct=('apy_cap_pct', 'mean'),
    best_season=('season', 'max')
).reset_index()

# Filter to TEs with at least 2 seasons
te_player_agg = te_player_agg[te_player_agg['total_seasons'] >= 2].copy()

# Top 10 bargains
top_bargains = te_player_agg.nlargest(10, 'avg_value')

print("Top 10 TE Bargains (Career Avg Value):")
print(top_bargains[['player_name', 'draft_round', 'draft_number', 'avg_value', 'total_seasons']])

# Bar chart
fig = px.bar(
    top_bargains,
    x='avg_value',
    y='player_name',
    color='draft_round',
    orientation='h',
    title='<b>Top 10 TE Bargains</b><br><sub>Highest Career Average Value Score (Min 2 Seasons)</sub>',
    labels={'avg_value': 'Avg Value Score', 'player_name': '', 'draft_round': 'Draft Round'},
    hover_data=['draft_number', 'total_seasons'],
    color_continuous_scale='Greens'
)

fig.update_layout(
    width=900,
    height=600,
    yaxis={'categoryorder': 'total ascending'}
)

fig.show()

# Save visualization
fig.write_image(output_dir / 'top_bargains.png', width=900, height=600)

---
## 6. Top 10 TE Busts (First-Round Disappointments)

In [ ]:
# Filter to first-round TEs
first_round_tes = te_player_agg[te_player_agg['draft_round'] == 1].copy()

# Bottom 10 by value (or all if fewer than 10)
top_busts = first_round_tes.nsmallest(min(10, len(first_round_tes)), 'avg_value')

print("First-Round TE Busts (Career Avg Value):")
print(top_busts[['player_name', 'draft_number', 'avg_value', 'total_seasons']])

# Bar chart
fig = px.bar(
    top_busts,
    x='avg_value',
    y='player_name',
    color='draft_number',
    orientation='h',
    title='<b>First-Round TE Busts</b><br><sub>Lowest Career Average Value Score</sub>',
    labels={'avg_value': 'Avg Value Score', 'player_name': '', 'draft_number': 'Draft Pick #'},
    hover_data=['total_seasons'],
    color_continuous_scale='Reds'
)

fig.update_layout(
    width=900,
    height=600,
    yaxis={'categoryorder': 'total descending'}
)

fig.show()

# Save visualization
fig.write_image(output_dir / 'top_busts.png', width=900, height=600)

---
## 7. Red Zone Efficiency vs. Salary (TE-Specific)

In [ ]:
# Load weekly stats to calculate touchdowns per snap (red zone efficiency proxy)
weekly = pd.read_parquet(DATA_DIR / 'weekly.parquet', engine='fastparquet')

# Filter to TEs and calculate TD rate
te_weekly = weekly[weekly['position'] == 'TE'].copy()
te_weekly['td_per_snap'] = te_weekly['touchdowns'] / te_weekly['snaps'].replace(0, np.nan)

# Aggregate to season level
te_td_stats = te_weekly.groupby(['player_id', 'season']).agg(
    total_tds=('touchdowns', 'sum'),
    total_snaps=('snaps', 'sum')
).reset_index()

te_td_stats['td_per_snap'] = te_td_stats['total_tds'] / te_td_stats['total_snaps'].replace(0, np.nan)

# Merge with TE scored data
te_redzone = te_scored.merge(te_td_stats, on=['player_id', 'season'], how='left')

# Filter to players with valid TD data
te_redzone = te_redzone[te_redzone['td_per_snap'].notna()].copy()

print(f"TEs with red zone data: {te_redzone.shape[0]:,} player-seasons")

# Scatter plot: TD rate vs. salary
fig = px.scatter(
    te_redzone,
    x='td_per_snap',
    y='apy_cap_pct',
    color='value_score',
    hover_name='player_name',
    hover_data=['season', 'total_tds', 'total_snaps'],
    title='<b>TE Red Zone Efficiency vs. Salary</b><br><sub>Touchdowns Per Snap vs. Cap Hit</sub>',
    labels={
        'td_per_snap': 'Touchdowns Per Snap',
        'apy_cap_pct': 'Salary (% of Cap)',
        'value_score': 'Value Score'
    },
    color_continuous_scale='RdYlGn',
    color_continuous_midpoint=0
)

fig.update_layout(
    width=1000,
    height=600
)

fig.show()

# Save visualization
fig.write_image(output_dir / 'redzone_vs_salary.png', width=1000, height=600)

---
## 8. Blocking vs. Receiving TE Value Comparison (TE-Specific)

In [ ]:
# Classify TEs as blocking vs. receiving based on targets per game
# Threshold: 4+ targets/game = receiving TE, <4 = blocking TE

# Calculate targets per game from weekly data
te_targets = te_weekly.groupby(['player_id', 'season']).agg(
    total_targets=('targets', 'sum'),
    games=('week', 'nunique')
).reset_index()

te_targets['targets_per_game'] = te_targets['total_targets'] / te_targets['games']

# Merge with scored data
te_role = te_scored.merge(te_targets, on=['player_id', 'season'], how='left')

# Classify role
te_role['role'] = te_role['targets_per_game'].apply(
    lambda x: 'Receiving TE' if x >= 4 else 'Blocking TE' if pd.notna(x) else 'Unknown'
)

# Filter to known roles
te_role = te_role[te_role['role'] != 'Unknown'].copy()

print(f"TEs with role classification: {te_role.shape[0]:,} player-seasons")
print(f"\nRole distribution:")
print(te_role['role'].value_counts())

# Compare average value by role
role_comparison = te_role.groupby('role').agg(
    avg_value=('value_score', 'mean'),
    median_value=('value_score', 'median'),
    avg_performance=('performance_zscore', 'mean'),
    avg_salary_pct=('apy_cap_pct', 'mean'),
    count=('value_score', 'size')
).reset_index()

print("\nBlocking vs. Receiving TE Value:")
print(role_comparison)

# Bar chart comparison
fig = go.Figure()

fig.add_trace(go.Bar(
    x=role_comparison['role'],
    y=role_comparison['avg_value'],
    text=role_comparison['avg_value'].round(2),
    textposition='outside',
    marker_color=['#2ca02c' if v > 0 else '#d62728' for v in role_comparison['avg_value']],
    hovertemplate='<b>%{x}</b><br>Avg Value: %{y:.2f}<br>N=%{customdata:,}<extra></extra>',
    customdata=role_comparison['count']
))

fig.update_layout(
    title='<b>Blocking vs. Receiving TE Value Comparison</b><br><sub>Average Value Score by TE Role</sub>',
    xaxis_title='',
    yaxis_title='Average Value Score',
    width=700,
    height=500,
    showlegend=False
)

fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
fig.show()

# Save visualization
fig.write_image(output_dir / 'blocking_vs_receiving.png', width=700, height=500)

---
## 9. Summary Statistics and Key Findings

In [ ]:
print("="*80)
print("TIGHT END MARKET INEFFICIENCY - KEY FINDINGS")
print("="*80)

print("\n1. CAREER ARC:")
print(f"Peak Years: Years {career_arc.nlargest(3, 'avg_value')['years_since_draft'].tolist()}")
print(f"Decline Begins: Year {career_arc[career_arc['years_since_draft'] >= 3].nsmallest(1, 'avg_value')['years_since_draft'].iloc[0] if len(career_arc) > 3 else 'N/A'}")

print("\n2. DRAFT ROI BY ROUND:")
print(te_draft_roi[['draft_round', 'avg_value', 'count']])

print("\n3. TOP 5 BARGAINS:")
print(top_bargains.head(5)[['player_name', 'draft_round', 'draft_number', 'avg_value']])

if len(top_busts) > 0:
    print("\n4. TOP FIRST-ROUND BUSTS:")
    print(top_busts.head(5)[['player_name', 'draft_number', 'avg_value']])

print("\n5. ROLE COMPARISON:")
print(role_comparison[['role', 'avg_value', 'avg_performance', 'count']])

print("\n" + "="*80)
print("CONCLUSION: Round 5 TEs provide exceptional value. Receiving TEs outperform")
print("blocking TEs. First-round TEs have high bust rate. Elite red zone efficiency")
print("justifies premium salaries for top-tier TEs like Kelce and Andrews.")
print("="*80)